In this notebook we will look at clustering and topic modeling. Both of these techniques can be used to explore data sets and potentially to improve other analyses (like predicting upvotes).

In [1]:
import scala.collection.immutable.TreeSet
import java.util

In [2]:
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/annotate-1.0.jar
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/opennlp-tools-1.6.0.jar
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/models.jar

Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/annotate-1.0.jar
Finished download of annotate-1.0.jar
Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/opennlp-tools-1.6.0.jar
Finished download of opennlp-tools-1.6.0.jar
Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/models.jar
Finished download of models.jar


In [3]:
import nlp.spark.annotate.{ Annotation, Annotator, Document }

In [4]:
import org.apache.spark.ml.classification._
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.{ Pipeline, PipelineModel, Transformer }
import org.apache.spark.ml.regression._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

In [5]:
val dataframe = sqlContext.read.parquet("stories.parquet").cache()

In [6]:
dataframe.count()

1226

In [7]:
dataframe

[docName: string, text: string, metadata: map<string,string>, annotations: array<struct<aType:string,begin:int,end:int,metadata:map<string,string>>>, isTopLevel: double, textLength: int, avgSentenceLengths: double, words: array<string>, numEntities: double, score: double, isStoryPred: string, popularity: string]

Now let's generate the same features as before

In [8]:
var hashingTF = new HashingTF()
hashingTF = hashingTF.setInputCol("words")
hashingTF = hashingTF.setOutputCol("wordfeatures")
hashingTF = hashingTF.setNumFeatures(25)

In [9]:
var idf = new IDF()
idf = idf.setInputCol("wordfeatures")
idf = idf.setOutputCol("tfidffeatures")

First we will try K-Means clustering. The clusters tend to be the same size which likely means there will be very little genre information.  
For more information see https://en.wikipedia.org/wiki/K-means_clustering

The basics of K-Means are that it finds k points (centroids), and assigns points to clusters based on which centroid each point is closest to. The points are fit to minimize sum of the square distance of points from their centroid.

In [10]:
var kmeans = new KMeans()
kmeans = kmeans.setFeaturesCol("tfidffeatures")
kmeans = kmeans.setPredictionCol("cluster")
kmeans = kmeans.setSeed(123)
kmeans = kmeans.setK(5)

// params with their default values
kmeans = kmeans.setMaxIter(20)
kmeans = kmeans.setTol(1.0E-4)

In [11]:
val pipeline = new Pipeline().setStages(Array(
  hashingTF,
  idf,
  kmeans
))

In [12]:
val kmeansModel = pipeline.fit(dataframe)

In [13]:
val kmeansDF = kmeansModel.transform(dataframe).cache()

In [14]:
kmeansDF

[docName: string, text: string, metadata: map<string,string>, annotations: array<struct<aType:string,begin:int,end:int,metadata:map<string,string>>>, isTopLevel: double, textLength: int, avgSentenceLengths: double, words: array<string>, numEntities: double, score: double, isStoryPred: string, popularity: string, wordfeatures: vector, tfidffeatures: vector, cluster: int]

(Assuming this test was performed with default values on the sample dataset)  
As we can see, this did _not_ result in roughly equal sized clusters.

In [15]:
%%dataframe
kmeansDF.groupBy("cluster").agg(count("docName"))

cluster count(docName) 0 341 1 557 2 51 3 227 4 113

Now let's see what are the special vocabularies of each cluster. To look at this we need to find, for each cluster, the words which most tend to appear in the given cluster versus the others.

So for each cluster $c$ we look at the top-$n$ words sorted by the following formula
$$p_{c,t} - \frac{\sum_{c'}{p_{c',t}}}{k}$$
where $p_{c,t}$ is the proportion occurrences of $t$ in $c$

In [16]:
val wordsForClusters = kmeansDF.selectExpr(
  // this will produce a table for each non stop word token occurrence with the name of the document it occurs in, 
  // and the cluster of document it occurs in
  "explode(words) as word", 
  "docName",
  "case when cluster = 0 then 1 else 0 end as c0", 
  "case when cluster = 1 then 1 else 0 end as c1", 
  "case when cluster = 2 then 1 else 0 end as c2", 
  "case when cluster = 3 then 1 else 0 end as c3", 
  "case when cluster = 4 then 1 else 0 end as c4"
).groupBy("word").agg( // now we group by the token occurrences
  count("docName").as("count"), // this is the number of occurrences of the token, the sum from above
  avg("c0").as("avg_c0"), // these are proportion of occurrences that are in a given cluster, p_c,t from above
  avg("c1").as("avg_c1"),
  avg("c2").as("avg_c2"),
  avg("c3").as("avg_c3"),
  avg("c4").as("avg_c4")
).filter(col("count") >= 50).selectExpr( // now we select words that have occurred at least 50 times
  "word",
  "count",
  "avg_c0 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c0", // this is the formula from above
  "avg_c1 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c1",
  "avg_c2 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c2",
  "avg_c3 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c3",
  "avg_c4 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c4"
)

In [17]:
(0 to 4).foreach {
  cluster =>
    println("=" * 50)
    wordsForClusters.orderBy(desc(s"c$cluster")).select("word", s"c$cluster", "count").take(20).foreach(println)
}
println("=" * 50)

[store,0.27058823529411763,51]
[joke,0.253125,64]
[smell,0.24329896907216492,97]
[crash,0.21999999999999997,50]
[pale,0.21509433962264152,53]
[wife,0.20277777777777778,144]
[below,0.19344262295081965,61]
[cheek,0.19189189189189187,74]
[rose,0.1902439024390244,82]
[babi,0.18888888888888888,72]
[sister,0.18888888888888888,54]
[nice,0.1867924528301887,106]
[memori,0.18636363636363637,132]
[sudden,0.18181818181818182,55]
[armi,0.17254901960784313,51]
[organ,0.17037037037037037,54]
[dad,0.16567164179104477,134]
[shake,0.16559139784946236,93]
[beat,0.16363636363636364,77]
[mother,0.16309523809523807,168]
[prompt,0.2852941176470588,68]
[post,0.2473684210526316,76]
[kiss,0.09999999999999998,90]
[written,0.09090909090909088,55]
[haven't,0.059259259259259234,54]
[write,0.05806451612903224,186]
[edit,0.05396825396825394,63]
[twist,0.0456140350877193,57]
[com,0.032,125]
[stori,0.025641025641025633,195]
[wp,0.02388059701492537,67]
[http,0.015277777777777779,144]
[fix,0.006896551724137917,58]
[sad,-

Now we will use LDA (Latent Dirichlet Allocation). LDA is a proper topic model. Topic models are a family of clustering algorithms used for examining text data.  

An interesting and important difference between LDA and K-Means is that each document is assigned a distribution over the topics, and each topic has a distribution over the vocabulary. Due to us using the hashing trick we can't look at the vocabulary distributions (since we can only see the hashes), but we can see the per-document distributions.

For more information  
https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation  
https://en.wikipedia.org/wiki/Topic_model  

In [18]:
var lda = new LDA()
lda = lda.setFeaturesCol("tfidffeatures")
lda = lda.setTopicDistributionCol("topics")
lda = lda.setSeed(123)
lda = lda.setK(5)

// params with their default values
// lda = lda.setDocConcentration(...) // the prior distribution over topics for documents, by default not set
// lda = lda.setTopcConcentration(...) // the prior distribution over topics for terms, by default not set
// lda = lda.setMaxIter(...)
lda = lda.setSubsamplingRate(0.05)
lda = lda.setOptimizer("online")

In [19]:
val pipeline2 = new Pipeline().setStages(Array(
  hashingTF,
  idf,
  lda
))

In [20]:
val model2 = pipeline2.fit(dataframe)

In [21]:
val ldaDF = model2.transform(dataframe).cache()

In [22]:
ldaDF

[docName: string, text: string, metadata: map<string,string>, annotations: array<struct<aType:string,begin:int,end:int,metadata:map<string,string>>>, isTopLevel: double, textLength: int, avgSentenceLengths: double, words: array<string>, numEntities: double, score: double, isStoryPred: string, popularity: string, wordfeatures: vector, tfidffeatures: vector, topics: vector]

In [23]:
val vectorToArray = sqlContext.udf.register(
  "vectorToArray", 
  (x:org.apache.spark.mllib.linalg.DenseVector) => x.toArray
)

In [25]:
ldaDF.selectExpr(
  "mean(vectorToArray(topics)[0])", 
  "mean(vectorToArray(topics)[1])", 
  "mean(vectorToArray(topics)[2])", 
  "mean(vectorToArray(topics)[3])", 
  "mean(vectorToArray(topics)[4])"
).collect().foreach {
  row =>
    row.toSeq.map(_.asInstanceOf[Double]).zipWithIndex.foreach {
      case (meanTopicValue, topic) =>
        println(f"topic $topic%-5d $meanTopicValue%-5.4f")
    }
}

topic 0     0.1388
topic 1     0.0183
topic 2     0.2996
topic 3     0.0294
topic 4     0.5108


We will perform a similar procedure as before, except now, each document, and so each token occurrence has a distribution over the topics.

So for each topic $T$ we look at the top-$n$ words sorted by the following formula
$$q_{T,t} - \frac{\sum_{T'}{q_{T',t}}}{k}$$
where $q_{T,t}$ is the average value each occurrence of $t$ has for $T$

In [26]:
val wordsForClusters2 = ldaDF.selectExpr(
  // this will produce a table for each non stop word token occurrence with the name of the document it occurs in, 
  // and the distribution of topics of the document
  "explode(words) as word", 
  "docName",
  "vectorToArray(topics)[0] as c0", // these are 'how much' the document of this occurrence is on the given topic
  "vectorToArray(topics)[1] as c1",
  "vectorToArray(topics)[2] as c2",
  "vectorToArray(topics)[3] as c3",
  "vectorToArray(topics)[4] as c4"
).groupBy("word").agg(
  count("docName").as("count"),
  avg("c0").as("avg_c0"), // these are the value of occurrences for a given topic, q_c,t from above
  avg("c1").as("avg_c1"),
  avg("c2").as("avg_c2"),
  avg("c3").as("avg_c3"),
  avg("c4").as("avg_c4")
).filter(col("count") >= 50).selectExpr(
  "word",
  "count",
  "avg_c0 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c0", // this is the formula from above
  "avg_c1 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c1",
  "avg_c2 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c2",
  "avg_c3 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c3",
  "avg_c4 - (avg_c0 + avg_c1 + avg_c2 + avg_c3 + avg_c4) / 5 as c4"
)

You may be able to seem some pattern in the vocabularies, but this is really the beginning of topic modeling. Feel free to try to improve the "meaningfulness" of the topics.

In [28]:
(0 to 4).foreach {
  cluster =>
    println("=" * 50)
    wordsForClusters2.orderBy(desc(s"c$cluster")).select("word", s"c$cluster", "count").take(20).foreach(println)
}
println("=" * 50)

[writingprompt,0.684807951506161,94]
[www,0.6465512670481768,107]
[lt,0.6139900174736745,83]
[com,0.6063480260424112,125]
[reddit,0.6062398777908435,115]
[wp,0.5903469381985431,67]
[http,0.5807779904698782,144]
[comment,0.5173188631908268,119]
[gt,0.4526672045780159,137]
[chuck,0.38415348845714026,56]
[cigarett,0.18301940029451774,59]
[write,0.18031743627464486,186]
[jake,0.1596124225188428,50]
[prompt,0.13715998149477365,68]
[smoke,0.10370957437754638,84]
[murder,0.06724655392927575,57]
[gonna,0.0659546796634535,63]
[post,0.05640925205210659,76]
[jack,0.05353889480137061,84]
[bathroom,0.047840654340307875,58]
[prompt,-0.186395311510402,68]
[post,-0.18779209463720356,76]
[write,-0.19175194371705281,186]
[share,-0.19229124190268704,75]
[written,-0.19244088918202637,55]
[you'v,-0.19312041506301234,83]
[forget,-0.19322264627107763,60]
[born,-0.1932853069122535,64]
[twist,-0.1934305790593646,57]
[stori,-0.1936124793272916,195]
[read,-0.19380981226159075,195]
[kiss,-0.1939560813784808,90]
[

### Exercise 3. Use clustering to improve popularity prediction

Try and use clustering to improve your score prediction. You can feel free to use regression against the score, or classification using the binary target we made in notebook #1. Again, because of the small amount of data, it may be difficult to see a significant change.

In [29]:
val Array(trainScoreDF, testScoreDF) = ldaDF.randomSplit(Array(0.75, 0.25), 123)

In [30]:
var labelIndexerScore = new StringIndexer()
labelIndexerScore = labelIndexerScore.setInputCol("popularity")
labelIndexerScore = labelIndexerScore.setOutputCol("popularityLabel")
val labelIndexScoreModel = labelIndexerScore.fit(trainScoreDF)
labelIndexScoreModel.labels.toList

List(not-popular, popular)

In [31]:
var vectorAssemblerScore = new VectorAssembler()
vectorAssemblerScore = vectorAssemblerScore.setInputCols(Array(
  "textLength",
  "avgSentenceLengths", 
  "numEntities",
  "tfidffeatures",
  "topics"
))
vectorAssemblerScore = vectorAssemblerScore.setOutputCol("features")

var classifierScore = new DecisionTreeClassifier()
classifierScore = classifierScore.setFeaturesCol("features")
classifierScore = classifierScore.setLabelCol("popularityLabel")

var labelConverterScore = new IndexToString()
labelConverterScore = labelConverterScore.setInputCol("prediction")
labelConverterScore = labelConverterScore.setOutputCol("popularityPred")
labelConverterScore = labelConverterScore.setLabels(labelIndexScoreModel.labels)

var pipelineScore = new Pipeline().setStages(Array(
  labelIndexerScore,
  vectorAssemblerScore,
  classifierScore,
  labelConverterScore
))

In [32]:
val modelScore = pipelineScore.fit(trainScoreDF)

In [33]:
val predictionsScore = modelScore.transform(testScoreDF)

In [34]:
predictionsScore

[docName: string, text: string, metadata: map<string,string>, annotations: array<struct<aType:string,begin:int,end:int,metadata:map<string,string>>>, isTopLevel: double, textLength: int, avgSentenceLengths: double, words: array<string>, numEntities: double, score: double, isStoryPred: string, popularity: string, wordfeatures: vector, tfidffeatures: vector, topics: vector, popularityLabel: double, features: vector, rawPrediction: vector, probability: vector, prediction: double, popularityPred: string]

In [35]:
var evaluatorScore = new BinaryClassificationEvaluator()
evaluatorScore = evaluatorScore.setLabelCol("popularityLabel")
evaluatorScore = evaluatorScore.setRawPredictionCol("rawPrediction")
val auc = evaluatorScore.evaluate(predictionsScore)
println("AUC = " + auc)

AUC = 0.4978759398496241


In [36]:
predictionsScore.select("popularityPred", "popularity").rdd.map {
  row =>
    s"truth: ${row.getString(1)}, prediction: ${row.getString(0)}"
}.groupBy(identity).mapValues(_.size).collect().foreach(println)

(truth: not-popular, prediction: not-popular,139)
(truth: not-popular, prediction: popular,36)
(truth: popular, prediction: not-popular,113)
(truth: popular, prediction: popular,39)


### end of exercise